In [ ]:
import sys
import cv2
import numpy as np
import skimage
from PyQt5.QtWidgets import QApplication, QMainWindow, QPushButton, QLabel, QVBoxLayout, QHBoxLayout, QWidget, \
    QFileDialog, QGridLayout, QSlider, QComboBox, QFormLayout
from PyQt5.QtGui import QPixmap, QImage
from PyQt5.QtCore import Qt
from matplotlib import pyplot as plt
from io import BytesIO
from scipy.ndimage import binary_fill_holes


class ImageProcessingToolbox(QMainWindow):
    def __init__(self):
        super().__init__()
        self.noisy_image = None
        self.noise_applied = False
        self.input_image = None
        self.initUI()

    def initUI(self):
        # Set up the main window title, size, and other UI elements like labels and buttons
        self.setWindowTitle('Image Processing Toolbox')
        self.setGeometry(200, 100, 1600, 1000)

        self.input_image_label = QLabel(self)
        self.output_image_label = QLabel(self)

        self.hist_input_label = QLabel(self)
        self.hist_output_label = QLabel(self)

        self.load_button = QPushButton('Load Image', self)
        self.load_button.clicked.connect(self.load_image)
        self.save_botton = QPushButton('Save Image', self)
        self.save_botton.clicked.connect(self.save_image)
        self.create_basic_operations_group()
        self.create_image_transform_group()
        self.create_noise_filter_group()

        self.control_layout = QHBoxLayout()
        self.control_layout.addWidget(self.load_button)
        self.control_layout.addWidget(self.save_botton)
        self.control_layout.addWidget(self.basic_operations_button)
        self.control_layout.addWidget(self.image_transform_button)
        self.control_layout.addWidget(self.noise_filter_button)

        self.image_layout = QGridLayout()
        self.image_layout.addWidget(QLabel("Input Image:"), 0, 0)
        self.image_layout.addWidget(self.input_image_label, 1, 0)
        self.image_layout.addWidget(QLabel("Input Histogram:"), 2, 0)
        self.image_layout.addWidget(self.hist_input_label, 3, 0)
        self.image_layout.addWidget(QLabel("Output Image:"), 0, 1)
        self.image_layout.addWidget(self.output_image_label, 1, 1)
        self.image_layout.addWidget(QLabel("Output Histogram:"), 2, 1)
        self.image_layout.addWidget(self.hist_output_label, 3, 1)

        self.main_layout = QVBoxLayout()
        self.main_layout.addLayout(self.image_layout)
        self.main_layout.addLayout(self.control_layout)

        # Add placeholder blank containers when initializing the interface
        self.placeholder = QWidget(self)
        self.placeholder.setFixedSize(400, 200)  # Set size of placeholder
        self.main_layout.addWidget(self.placeholder)

        container = QWidget()
        container.setLayout(self.main_layout)
        self.setCentralWidget(container)

    def create_basic_operations_group(self):
        # Create a group of buttons for basic image operations like grayscale conversion, binary thresholding, etc.
        self.basic_operations_group = QWidget()
        layout = QGridLayout()  # Apply grid layout

        # Create buttons
        grayscale_button = QPushButton('Grayscale', self)
        grayscale_button.clicked.connect(self.convert_to_grayscale)
        binary_button = QPushButton('Binary Threshold', self)
        binary_button.clicked.connect(self.convert_to_binary)
        edge_button = QPushButton('Edge Detection', self)
        edge_button.clicked.connect(self.edge_detection)
        face_detection_button = QPushButton('Face Detection', self)
        face_detection_button.clicked.connect(self.face_detection)
        sharpen_button = QPushButton('Sharpen', self)
        sharpen_button.clicked.connect(self.sharpen)
        hole_filling_button = QPushButton('Hole Filling', self)
        hole_filling_button.clicked.connect(self.fill_holes)

        # Set the button size to be uniform
        buttons = [grayscale_button, binary_button, edge_button, face_detection_button, sharpen_button,
                   hole_filling_button]
        for button in buttons:
            button.setFixedSize(280, 40)  # Set a uniform size

        # Apply buttons to grid layout
        layout.addWidget(grayscale_button, 0, 0)
        layout.addWidget(binary_button, 0, 1)
        layout.addWidget(edge_button, 1, 0)
        layout.addWidget(face_detection_button, 1, 1)
        layout.addWidget(sharpen_button, 2, 0)
        layout.addWidget(hole_filling_button, 2, 1)

        self.basic_operations_group.setLayout(layout)
        self.basic_operations_group.setVisible(False)

        self.basic_operations_button = QPushButton("Basic Operations", self)
        self.basic_operations_button.clicked.connect(self.toggle_basic_operations)

    def create_image_transform_group(self):
        # Create a group of buttons for image transformations like gamma, log, and negative transformations
        self.image_transform_group = QWidget()
        layout = QGridLayout()  # Apply grid layout

        gamma_button = QPushButton('Gamma Transformation', self)
        gamma_button.clicked.connect(self.gamma_transformation)
        log_trans_button = QPushButton('Log Transformation', self)
        log_trans_button.clicked.connect(self.log_transformation)
        neg_trans_button = QPushButton('Negative Transformation', self)
        neg_trans_button.clicked.connect(self.negative_transformation)
        hist_eq_button = QPushButton('Histogram Equalization', self)
        hist_eq_button.clicked.connect(self.histogram_equalization)

        # Set the button size to be uniform
        buttons = [gamma_button, hist_eq_button, log_trans_button, neg_trans_button]
        for button in buttons:
            button.setFixedSize(280, 40)  # Set a uniform size

        layout.addWidget(gamma_button, 0, 0)
        layout.addWidget(log_trans_button, 0, 1)
        layout.addWidget(neg_trans_button, 1, 0)
        layout.addWidget(hist_eq_button, 1, 1)

        self.image_transform_group.setLayout(layout)
        self.image_transform_group.setVisible(False)

        self.image_transform_button = QPushButton("Image Transformations", self)
        self.image_transform_button.clicked.connect(self.toggle_image_transform)

    def create_noise_filter_group(self):
        # Create a group of controls for adding noise and filters to an image
        self.noise_filter_group = QWidget()
        layout = QFormLayout()

        self.noise_combo = QComboBox()
        self.noise_combo.addItem("Pepper")
        self.noise_combo.addItem("Salt")
        self.noise_combo.addItem("Salt and Pepper")
        self.noise_combo.addItem("Gaussian")
        layout.addRow("Noise Type:", self.noise_combo)

        self.noise_slider = QSlider()
        self.noise_slider.setOrientation(Qt.Horizontal)
        self.noise_slider.setMinimum(1)
        self.noise_slider.setMaximum(20)
        self.noise_slider.setValue(1)
        layout.addRow("Noise Intensity:", self.noise_slider)

        self.filter_combo = QComboBox()
        self.filter_combo.addItem("Blur")
        self.filter_combo.addItem("Mean")
        self.filter_combo.addItem("Gaussian")
        self.filter_combo.addItem("Median")
        self.filter_combo.addItem("Bilateral")
        layout.addRow("Filter Type:", self.filter_combo)

        self.filter_slider = QSlider()
        self.filter_slider.setOrientation(Qt.Horizontal)
        self.filter_slider.setMinimum(1)
        self.filter_slider.setMaximum(20)
        self.filter_slider.setValue(1)
        layout.addRow("Filter Intensity:", self.filter_slider)

        apply_button = QPushButton("Apply")
        apply_button.clicked.connect(self.apply_noise_and_filter)
        apply_button.setFixedSize(280, 40)
        layout.addWidget(apply_button)

        self.noise_filter_group.setLayout(layout)
        self.noise_filter_group.setVisible(False)

        self.noise_filter_button = QPushButton("Noise and Filter", self)
        self.noise_filter_button.clicked.connect(self.toggle_noise_filter)

    def toggle_basic_operations(self):
        # Toggle the visibility of the basic operations group and hide other groups
        self.hide_all_groups()
        self.basic_operations_group.setVisible(not self.basic_operations_group.isVisible())
        if self.basic_operations_group.isVisible():
            self.main_layout.addWidget(self.basic_operations_group)
        self.placeholder.setVisible(False)  # Hide the placeholder

    def toggle_image_transform(self):
        # Toggle the visibility of the image transformations group and hide other groups
        self.hide_all_groups()
        self.image_transform_group.setVisible(not self.image_transform_group.isVisible())
        if self.image_transform_group.isVisible():
            self.main_layout.addWidget(self.image_transform_group)
        self.placeholder.setVisible(False)  # Hide the placeholder

    def toggle_noise_filter(self):
        # Toggle the visibility of the noise and filter group and hide other groups
        self.hide_all_groups()
        self.noise_filter_group.setVisible(not self.noise_filter_group.isVisible())
        if self.noise_filter_group.isVisible():
            self.main_layout.addWidget(self.noise_filter_group)
        self.placeholder.setVisible(False)  # Hide the placeholder

    def hide_all_groups(self):
        # Hide all the operation groups and show the placeholder widget
        self.basic_operations_group.setVisible(False)
        self.image_transform_group.setVisible(False)
        self.noise_filter_group.setVisible(False)
        self.placeholder.setVisible(True)  # Display the placeholder

    def load_image(self):
        # Load an image file using a file dialog and display it
        options = QFileDialog.Options()
        file_name, _ = QFileDialog.getOpenFileName(self, "Open Image File", "", "Images (*.png *.jpg *.jpeg *.bmp)",
                                                   options=options)
        if file_name:
            print("Selected file:", file_name)
            self.input_image = cv2.imread(file_name)

            if self.input_image is None:
                print("Error: Image could not be read.")
            else:
                self.display_image(self.input_image, self.input_image_label)
                self.display_histogram(self.input_image, self.hist_input_label)

    def display_image(self, image, label, fixed_width=600, fixed_height=500):
        # Display the given image in the specified label widget with a fixed size
        h, w, _ = image.shape
        # Adjust the size of visualizasion
        image = cv2.resize(image, (fixed_width, fixed_height))
        qimage = QImage(image.data, image.shape[1], image.shape[0], QImage.Format_RGB888).rgbSwapped()
        pixmap = QPixmap.fromImage(qimage)
        label.setPixmap(pixmap)

    def display_histogram(self, image, label):
        # Display the histogram of the given image in the specified label widget
        if len(image.shape) == 3:
            # Calculate the histograms for the three RGB color channels separately
            color = ('b', 'g', 'r')
            for i, col in enumerate(color):
                # Calculate the histogram for each color channel
                hist, bins = np.histogram(image[:, :, i].flatten(), 256, [0, 256])
                plt.hist(image[:, :, i].flatten(), 256, [0, 256], rwidth=0.8, color=col, alpha=0.5)
                plt.xlim([0, 256])
                plt.xticks(range(0, 256, 40))
            plt.xlabel('Intensity Value')
            plt.ylabel('Pixel Count')
            plt.title('Histogram')
        else:
            hist, bins = np.histogram(image.flatten(), 256, [0, 256])
            plt.hist(image.flatten(), 256, [0, 256], color='r')
            plt.xlim([0, 256])
            plt.xlabel('Intensity Value')
            plt.ylabel('Pixel Count')
            plt.title('Histogram')

        # Save the histogram plot to a BytesIO buffer
        buf = BytesIO()
        plt.savefig(buf, format='png')
        buf.seek(0)
        # Convert the buffer to a QImage and scale it down while maintaining the aspect ratio
        qimage = QImage.fromData(buf.getvalue())
        qimage = qimage.scaled(qimage.width() // 2, qimage.height() // 2, Qt.KeepAspectRatio)
        # Convert the QImage to a QPixmap and display it in the label widget
        pixmap = QPixmap.fromImage(qimage)
        label.setPixmap(pixmap)
        plt.close()

    def apply_noise_and_filter(self):
        # Apply noise and filter to the input image based on user selections
        noise_type = self.noise_combo.currentText().lower().replace(" ", "_")
        noise_intensity = self.noise_slider.value() / 100.0

        filter_type = self.filter_combo.currentText().lower().replace(" ", "_")
        filter_intensity = self.filter_slider.value()

        # Determine if the noise function has been used beforehand
        self.add_noise(self.input_image.copy(), self.output_image_label, noise_type=noise_type,
                       intensity=noise_intensity)
        self.noise_applied = True
        if self.noise_applied:
            self.apply_filter(self.noisy_image, self.output_image_label, filter_type=filter_type,
                              intensity=filter_intensity)
        else:
            self.apply_filter(self.input_image.copy(), self.output_image_label, filter_type=filter_type,
                              intensity=filter_intensity)

    def convert_to_grayscale(self):
        # Convert the input image to grayscale and display the result
        gray_image = cv2.cvtColor(self.input_image, cv2.COLOR_BGR2GRAY)
        self.display_image(cv2.cvtColor(gray_image, cv2.COLOR_GRAY2RGB), self.output_image_label)
        self.display_histogram(gray_image, self.hist_output_label)

    def convert_to_binary(self):
        # Convert the input image to a binary image using a threshold and display the result
        gray_image = cv2.cvtColor(self.input_image, cv2.COLOR_BGR2GRAY)
        _, binary_image = cv2.threshold(gray_image, 127, 255, cv2.THRESH_BINARY)
        self.display_image(cv2.cvtColor(binary_image, cv2.COLOR_GRAY2RGB), self.output_image_label)
        self.display_histogram(binary_image, self.hist_output_label)

    def apply_filter(self, image, label, filter_type='blur', intensity=5):
        # Apply a selected filter to the image and display the filtered image
        if filter_type == 'blur':
            filtered_image = cv2.boxFilter(image, -1, (intensity * 2 + 1, intensity * 2 + 1))
        elif filter_type == 'mean':
            filtered_image = cv2.blur(image, (intensity * 2 + 1, intensity * 2 + 1))
        elif filter_type == 'gaussian':
            filtered_image = cv2.GaussianBlur(image, (intensity * 2 + 1, intensity * 2 + 1), 0)
        elif filter_type == 'median':
            filtered_image = cv2.medianBlur(image, intensity * 2 + 1)
        elif filter_type == 'bilateral':
            filtered_image = cv2.bilateralFilter(image, intensity * 2 + 1, intensity * 2 + 1, intensity * 2 + 1)
        else:
            filtered_image = image

        self.display_image(filtered_image, label)
        self.display_histogram(filtered_image, self.hist_output_label)

    def add_noise(self, image, label, noise_type='gaussian', intensity=0.01):
        # Add noise to the image based on the selected noise type and intensity
        if noise_type == 'gaussian':
            row, col, ch = image.shape
            mean = 0
            sigma = intensity ** 0.5
            gauss = np.random.normal(mean, sigma, (row, col, ch))
            gauss = gauss.reshape(row, col, ch)
            noisy = image + gauss * 255
        elif noise_type == 'salt':
            noisy = np.copy(image)
            num_salt = np.ceil(intensity * image.size * 0.5)
            coords = [np.random.randint(0, i - 1, int(num_salt)) for i in image.shape]
            noisy[coords[0], coords[1], :] = 255
        elif noise_type == 'pepper':
            noisy = np.copy(image)
            num_pepper = np.ceil(intensity * image.size * 0.5)
            coords = [np.random.randint(0, i - 1, int(num_pepper)) for i in image.shape]
            noisy[coords[0], coords[1], :] = 0
        elif noise_type == 'salt_and_pepper':
            noisy = np.copy(image)
            num_salt = np.ceil(intensity * image.size * 0.5)
            num_pepper = np.ceil(intensity * image.size * 0.5)
            coords_salt = [np.random.randint(0, i - 1, int(num_salt)) for i in image.shape]
            coords_pepper = [np.random.randint(0, i - 1, int(num_pepper)) for i in image.shape]
            noisy[coords_salt[0], coords_salt[1], :] = 255
            noisy[coords_pepper[0], coords_pepper[1], :] = 0
        else:
            noisy = image

        noisy = np.clip(noisy, 0, 255).astype(np.uint8)
        self.noisy_image = noisy
        self.display_image(noisy, label)
        self.display_histogram(noisy, self.hist_output_label)

    def fill_holes(self):
        # Fill holes in a binary image using morphological operations and display the result
        gray_image = cv2.cvtColor(self.input_image, cv2.COLOR_BGR2GRAY)
        _, binary_image = cv2.threshold(gray_image, 127, 255, cv2.THRESH_BINARY)
        filled_image = binary_fill_holes(binary_image).astype(np.uint8) * 255
        self.display_image(cv2.cvtColor(filled_image, cv2.COLOR_GRAY2RGB), self.output_image_label)
        self.display_histogram(filled_image, self.hist_output_label)

    def edge_detection(self):
        # Perform edge detection on the input image using the Canny algorithm and display the result
        gray_image = cv2.cvtColor(self.input_image, cv2.COLOR_BGR2GRAY)
        edges = cv2.Canny(gray_image, 100, 200)
        self.display_image(cv2.cvtColor(edges, cv2.COLOR_GRAY2RGB), self.output_image_label)
        self.display_histogram(edges, self.hist_output_label)

    def face_detection(self):
        # Detect faces in the input image using Haar cascades and display the result with rectangles around faces
        gray_image = cv2.cvtColor(self.input_image, cv2.COLOR_BGR2GRAY)
        face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        faces = face_cascade.detectMultiScale(gray_image, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
        output_image = self.input_image.copy()
        face_count = 0
        for (x, y, w, h) in faces:
            cv2.rectangle(output_image, (x, y), (x + w, y + h), (255, 0, 0), 5)
            face_count += 1

        # Print the count of faces
        if faces is None:
            print("Detect no face")
        else:
            print(f"Detect {face_count} faces")
        self.display_image(output_image, self.output_image_label)
        self.display_histogram(output_image, self.hist_output_label)

    def sharpen(self):
        # Sharpen the input image using an unsharp masking technique and display the result
        kernel = np.array([[0, -1, 0],
                           [-1, 5, -1],
                           [0, -1, 0]])
        sharpened_image = cv2.filter2D(self.input_image, -1, kernel)
        self.display_image(sharpened_image, self.output_image_label)
        self.display_histogram(sharpened_image, self.hist_output_label)

    def gamma_transformation(self, gamma=1.0):
        # Apply gamma correction to the input image to adjust its brightness and contrast
        gamma = 0.5
        table = np.array([(i / 255.0) ** gamma * 255 for i in np.arange(0, 256)]).astype("uint8")
        gamma_corrected_image = cv2.LUT(self.input_image, table)
        self.display_image(gamma_corrected_image, self.output_image_label)
        self.display_histogram(gamma_corrected_image, self.hist_output_label)

    def histogram_equalization(self):
        # Perform histogram equalization on the input image to enhance its contrast
        if len(self.input_image.shape) == 2:
            equalized_image = cv2.equalizeHist(self.input_image)
        else:
            img_y_cr_cb = cv2.cvtColor(self.input_image, cv2.COLOR_BGR2YCrCb)
            y, cr, cb = cv2.split(img_y_cr_cb)
            y_eq = cv2.equalizeHist(y)
            equalized_image = cv2.merge((y_eq, cr, cb))
            equalized_image = cv2.cvtColor(equalized_image, cv2.COLOR_YCrCb2BGR)

        self.display_image(equalized_image, self.output_image_label)
        self.display_histogram(equalized_image, self.hist_output_label)

    def negative_transformation(self):
        # Invert the colors of the input image to create a negative effect and display the result
        negative_image = 255 - self.input_image
        self.display_image(negative_image, self.output_image_label)
        self.display_histogram(negative_image, self.hist_output_label)

    def log_transformation(self):
        # Apply a logarithmic transformation to the input image to adjust its contrast
        epsilon = 1e-5  # A small positive numbers to avoid division by zero errors in log transformation
        c = 255 / np.log(1 + np.max(self.input_image))
        log_transformed_image = c * (np.log(self.input_image + 1 + epsilon))
        log_transformed_image = np.array(log_transformed_image, dtype=np.uint8)
        self.display_image(log_transformed_image, self.output_image_label)
        self.display_histogram(log_transformed_image, self.hist_output_label)

    def save_image(self):
        # Save the processed image
        if self.output_image_label.pixmap() is not None:
            options = QFileDialog.Options()
            file_name, _ = QFileDialog.getSaveFileName(self, "Save Processed Image", "",
                                                       "Images (*.png *.jpg *.jpeg *.bmp)", options=options)
            if file_name:
                pixmap = self.output_image_label.pixmap()
                pixmap.save(file_name)
                print(f"Image saved to {file_name}")
            else:
                print("Save operation canceled.")
        else:
            print("No processed image to save.")


# Create the application, the main window, and start the event loop
if __name__ == '__main__':
    app = QApplication(sys.argv)
    ex = ImageProcessingToolbox()
    ex.show()
    sys.exit(app.exec_())
